**About** : This notebook is used to validate models.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Initialization

### Imports

In [ ]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
device = torch.cuda.get_device_name(0)
print(device)

In [ ]:
import os
import sys
import glob
import json
import timm
import torch
import operator
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import *
from collections import Counter
from numerize.numerize import numerize

pd.set_option('display.width', 500)
pd.set_option('max_colwidth', 100)

In [ ]:
from params import *
from util.plots import *
from util.metrics import *

from data.dataset import ContrailDataset
from data.transforms import get_transfos
from data.preparation import prepare_data
from util.plots import plot_sample
from util.logger import upload_to_kaggle

from inference_main import kfold_inference, Config

### Data

In [ ]:
df = prepare_data(DATA_PATH, processed_folder="false_color/", use_raw=True)

In [ ]:
if "fold" not in df.columns:
    folds = pd.read_csv(DATA_PATH + "folds_4.csv")
    df = df.merge(folds)

In [ ]:
df_val = df[df['fold'] == 0].reset_index(drop=True)

In [ ]:
dataset = ContrailDataset(df_val, get_transfos())

gts = []
for i in tqdm(range(len(dataset))):
    image, mask, _ = dataset[i]
    gts.append(mask[:1].numpy())
gts = np.array(gts)

### Inference

In [ ]:
USE_FP16 = True
SAVE = True

EXP_FOLDER = "../logs/YOUR_EXP_HERE/"

In [ ]:
preds = kfold_inference(df, EXP_FOLDER, use_fp16=USE_FP16, save=SAVE)

In [ ]:
preds_gpu = torch.from_numpy(preds[0]).cuda()
truths_gpu = torch.from_numpy(gts).cuda()

dices = {}
for th in tqdm(np.round(np.arange(0.2, 0.6, 0.01), 2)):
    dices[th] = dice_score_torch((preds_gpu > th), truths_gpu)
    
th, dice = max(dices.items(), key=operator.itemgetter(1))
print(f'-> CV dice :  {dice:.4f}   (th={th:.2f})')

### Blends

In [ ]:
EXP_FOLDERS = [
    "../logs/YOUR_EXP1_HERE/",
    "../logs/YOUR_EXP2_HERE/",
    "../logs/YOUR_EXP3_HERE/",
]

PREDS = {f: torch.from_numpy(np.load(f + "pred_val_0.npy")).cuda() for f in tqdm(EXP_FOLDERS)}

In [ ]:
truths_gpu = torch.from_numpy(gts).cuda()
scores = []

for exp_folder in EXP_FOLDERS:
    preds = PREDS[exp_folder]

    dices = {}
    for th in np.round(np.arange(0.2, 0.6, 0.01), 2):
        dices[th] = dice_score_torch((preds > th), truths_gpu)

    th, dice = max(dices.items(), key=operator.itemgetter(1))
    print(f'-> {exp_folder}\t-  CV dice :  {dice:.4f}   (th={th:.2f})')
    scores.append(dice)
    
print(f'\n  --> Avg of CV : {np.mean(scores):.4f} +/- {np.std(scores):.4f}')

preds = torch.stack([PREDS[f] for f in EXP_FOLDERS], 0).mean(0)

dices = {}
for th in np.round(np.arange(0.4, 0.6, 0.01), 2):
    dices[th] = dice_score_torch((preds > th), truths_gpu)
    
th, dice = max(dices.items(), key=operator.itemgetter(1))
print(f'  --> CV of avg : {dice:.4f}   (th={th:.2f})')

Done ! 